In [212]:
import plotly
import plotly.express as px
import pandas as pd
import geojson
pd.options.mode.chained_assignment = None 
from geojson_rewind import rewind
import apportionpy.apportionment as ap

In [213]:
df = pd.read_csv('Final_Data/PC01_C01_00.csv')
df.head()

,States,Pop_T,Pop_M,Pop_F,Urban_T,Urban_M,Urban_F,Rural_T,Rural_M,Rural_F,...,Jain_T,Jain_M,Jain_F,Other_T,Other_M,Other_F,NS_T,NS_M,NS_F,Seats
0,ANDAMAN & NICOBAR ISLANDS,356152.0,192972.0,163180.0,116198.0,64011.0,52187.0,239954.0,128961.0,110993.0,...,23.0,12.0,11.0,238.0,128.0,110.0,851.0,506.0,345.0,1.0
1,ANDHRA PRADESH,76210007.0,38527413.0,37682594.0,20808940.0,10590209.0,10218731.0,55401067.0,27937204.0,27463863.0,...,41846.0,21613.0,20233.0,4768.0,2450.0,2318.0,94934.0,49136.0,45798.0,42.0
2,ARUNACHAL PRADESH,1097968.0,579941.0,518027.0,227881.0,125261.0,102620.0,870087.0,454680.0,415407.0,...,216.0,130.0,86.0,337399.0,168613.0,168786.0,9302.0,4578.0,4724.0,2.0
3,ASSAM,26655528.0,13777037.0,12878491.0,3439240.0,1837092.0,1602148.0,23216288.0,11939945.0,11276343.0,...,23957.0,12840.0,11117.0,22999.0,11795.0,11204.0,11369.0,6017.0,5352.0,14.0
4,BIHAR,82998509.0,43243795.0,39754714.0,8681800.0,4648799.0,4033001.0,74316709.0,38594996.0,35721713.0,...,16085.0,8446.0,7639.0,52905.0,27336.0,25569.0,37817.0,20541.0,17276.0,40.0


In [214]:
methods = ["hamilton","adam","webster","jefferson","huntington hill"]
df.fillna(0)
seats = df.loc[len(df.index)-1].at['Seats']
populations = df['Pop_T'][:-2]
for i in range(len(methods)):
    result = ap.Apportion(seats=seats, populations=populations, method=methods[i]).fair_shares
    result.append(0)
    result.append(sum(result))
    df[methods[i]] = result
df.head()


,States,Pop_T,Pop_M,Pop_F,Urban_T,Urban_M,Urban_F,Rural_T,Rural_M,Rural_F,...,Other_F,NS_T,NS_M,NS_F,Seats,hamilton,adam,webster,jefferson,huntington hill
0,ANDAMAN & NICOBAR ISLANDS,356152.0,192972.0,163180.0,116198.0,64011.0,52187.0,239954.0,128961.0,110993.0,...,110.0,851.0,506.0,345.0,1.0,0,1,0,0,1
1,ANDHRA PRADESH,76210007.0,38527413.0,37682594.0,20808940.0,10590209.0,10218731.0,55401067.0,27937204.0,27463863.0,...,2318.0,94934.0,49136.0,45798.0,42.0,40,39,40,41,40
2,ARUNACHAL PRADESH,1097968.0,579941.0,518027.0,227881.0,125261.0,102620.0,870087.0,454680.0,415407.0,...,168786.0,9302.0,4578.0,4724.0,2.0,1,1,1,0,1
3,ASSAM,26655528.0,13777037.0,12878491.0,3439240.0,1837092.0,1602148.0,23216288.0,11939945.0,11276343.0,...,11204.0,11369.0,6017.0,5352.0,14.0,14,14,14,14,14
4,BIHAR,82998509.0,43243795.0,39754714.0,8681800.0,4648799.0,4033001.0,74316709.0,38594996.0,35721713.0,...,25569.0,37817.0,20541.0,17276.0,40.0,44,43,44,45,43


In [215]:
def voting_power(method , comm):
    vote_p = df[comm]/df['Pop_T'] * df[method]
    Total_seats = df.loc[len(df.index)-1].at[method]
    Pop_of_comm = df.loc[len(df.index)-1].at[comm]
    Pop_of_india = df.loc[len(df.index)-1].at['Pop_T']
    return sum(vote_p[:-2]) - Total_seats * Pop_of_comm/Pop_of_india

voting_power('hamilton','Urban_T')  
x = ["Seats", "hamilton","adam","webster","jefferson","huntington hill"]
def voting_pow_y(comm):
    ans = []
    for elem in x:
        ans.append(voting_power(elem,comm))
    return ans


In [216]:
community = ["Male" , "Female" , "Urban" , "Rural" , "Hindu" , "Muslim" , "Christian" , "Sikh" ]
community_code = ["Pop_M" , "Pop_F" , "Urban_T" , "Rural_T" , "Hindu_T" , "Muslim_T" , "Christian_T" , "Sikh_T" ]

y_Seats = []
y_hamilton = []
y_adam = []
y_webster = []
y_jefferson = []
y_huntington_hill = []
for i in range(len(community)):
    y_Seats.append(voting_power("Seats",community_code[i]))
    y_hamilton.append(voting_power("hamilton",community_code[i]))
    y_adam.append(voting_power("adam",community_code[i]))
    y_webster.append(voting_power("webster",community_code[i]))
    y_jefferson.append(voting_power("jefferson",community_code[i]))
    y_huntington_hill.append(voting_power("huntington hill",community_code[i]))

df1 = pd.DataFrame(list(zip(community, y_Seats, y_hamilton, y_adam, y_webster, y_jefferson, y_huntington_hill)),
                columns =['Community', 'Seats', 'hamilton', 'adam', 'webster', 'jefferson', 'huntington hill'])
df1.head()
fig = px.line(x = community , y = y_Seats , title = "Voting Power", labels = {"x" : "Community" , "y" : "Voting Power"})
fig.add_scatter(x = community , y = y_hamilton , name = 'hamilton')
fig.add_scatter(x = community , y = y_adam , name = 'adam')
fig.add_scatter(x = community , y = y_webster , name = 'webster')
fig.add_scatter(x = community , y = y_jefferson , name = 'jefferson')
fig.add_scatter(x = community , y = y_huntington_hill , name = 'huntington hill')
fig.show()



